In [ ]:
import speech_recognition as sr
from gtts import gTTS
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import UserMessage, AssistantMessage
from azure.core.credentials import AzureKeyCredential

# Azure Configuration
endpoint = "https://models.github.ai/inference"
model_name = "microsoft/Phi-4"
token = "my_token"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

recognizer = sr.Recognizer()
chat_history = []

print("Start speaking to your AI assistant. Say 'stop listening' to exit.\n")

while True:
    try:
        # Speech-to-Text
        with sr.Microphone() as source:
            print("Say something...")
            audio = recognizer.listen(source)
            user_input = recognizer.recognize_google(audio)
            print("You said:", user_input)

        # Exit condition
        if user_input.lower() == "stop listening":
            print("Exiting chat.")
            break

        # Add user message to chat history
        chat_history.append(UserMessage(content=user_input))

        # Send input to model
        response = client.complete(
            messages=chat_history,
            temperature=1.0,
            top_p=1.0,
            max_tokens=1000,
            model=model_name
        )

        # Get AI response
        response_text = response.choices[0].message.content
        print("Assistant:", response_text)

        # Add AI response to history
        chat_history.append(AssistantMessage(content=response_text))

        # Text-to-Speech
        language = 'en'
        tts = gTTS(text=response_text, lang=language, slow=False)
        filename = "response.mp3"
        tts.save(filename)
        os.system(f"start {filename}")

        # Delete the mp3 file after playback
        print("Cleaning up the audio file...")
        os.remove(filename)

    except sr.UnknownValueError:
        print("Sorry, I couldn't understand your speech.")
    except KeyboardInterrupt:
        print("Keyboard interrupt detected. Exiting chat.")
        break
    except Exception as e:
        print(f"An error occurred: {e}")
        break

In [ ]:
!pip uninstall pyttsx3

In [ ]:
!pip uninstall pytt